# Pytorch 数据处理和加载
在pytorch中，处理和加载数据是深度学习训练过程中的关键步骤

为了高效地处理数据，pytorch提供了`torch.utils.data.Dataset`和`torch.utils.data.DataLoader`两个类来帮助我们管理数据集、批量加载和数据增强等任务。

Pytorch数据处理与加载的介绍：
- 自定义 Dataset：通过继承 `torch.utils.data.Dataset` 类来加载自己的数据集；
- DataLoader：使用 `torch.utils.data.DataLoader` 来批量加载数据，支持多线程加载数据并进行数据打乱；
- 数据预处理与增强：使用 `torchvision.transforms` 来对图像数据进行预处理和增强操作，从而提高模型的泛化能力；
- 加载标准数据集：使用 `torchvision.datasets` 来加载常用的标准数据集，如 MNIST、CIFAR-10 等；
- 多个数据源：通过组合多个 Dataset 实例来处理来自不同来源的数据。

## 1. 自定义 Dataset
`torch.utils.data.Dataset` 是一个抽象类，用户需要继承它并实现 `__len__` 和 `__getitem__` 方法来创建自己的数据集。

- `__len__(self)`：返回数据集的大小；
- `__getitem__(self, idx)`：根据索引返回数据和标签。

假设有一个简单的csv文件，或一些列表数据，可以通过继承`Dataset`类来创建一个自定义数据集。以下是一个简单的示例：

In [2]:
import torch
from torch.utils.data import Dataset

# self-defined dataset class
class MyDataset(Dataset):
    def __init__(self, X_data, Y_data):
        """
        initialize the dataset with input and output data.
        :param X_data: input properties
        :param Y_data: target labels
        """
        self.X_data = X_data
        self.Y_data = Y_data

    def __len__(self):
        """
        return the length of the dataset
        :return: length of the dataset
        """
        return len(self.X_data)

    def __getitem__(self, idx):
        """
        return the input and output data at the given index
        :param idx: index of the data
        :return: input and output data at the given index
        """
        x = torch.tensor(self.X_data[idx], dtype=torch.float32)       # convert to tensor
        y = torch.tensor(self.Y_data[idx], dtype=torch.float32)       # convert to tensor
        return x, y


# Example usage
X_data = [[1, 2], [3, 4], [5, 6], [7, 8]]
Y_data = [[0], [1], [0], [1]]  # binary labels

# Create an instance of the dataset
dataset = MyDataset(X_data, Y_data)

## 2. 使用 DataLoader 加载数据
`torch.utils.data.DataLoader` 是pytorch提供的一个数据加载器，可以将数据集按批次（batch）加载，并支持多线程加载数据。它的主要参数包括：

In [3]:
from torch.utils.data import DataLoader

# Create a DataLoader, batch_size used to set the number of samples per batch
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# print the load data
for epoch in range(2):
    print(f"Epoch {epoch + 1}:")
    for batch_idx, (inputs, labels) in enumerate(dataloader):
        print(f"Batch {batch_idx + 1}:")
        print("Inputs:", inputs)
        print("Labels:", labels)

Epoch 1:
Batch 1:
Inputs: tensor([[1., 2.],
        [5., 6.]])
Labels: tensor([[0.],
        [0.]])
Batch 2:
Inputs: tensor([[3., 4.],
        [7., 8.]])
Labels: tensor([[1.],
        [1.]])
Epoch 2:
Batch 1:
Inputs: tensor([[3., 4.],
        [5., 6.]])
Labels: tensor([[1.],
        [0.]])
Batch 2:
Inputs: tensor([[7., 8.],
        [1., 2.]])
Labels: tensor([[1.],
        [0.]])


- `batch_size`：每次加载的样本数量；
- `shuffle`：是否打乱数据集，通常需要打乱数据；
- `drop_last`：如果数据集中的样本数不能被 `batch_size` 整除，设置为 `True` 则丢弃最后一个不完整的batch；

在每次循环中，`DataLoader` 会返回一个批次的数据，包括输入特征（inputs）和标签（labels）。可以使用 `for` 循环来遍历数据集，获取每个批次的数据。

## 3. 数据预处理与增强
pytorch提供了`torchvision.transforms`模块来对图像数据进行预处理和增强操作。常用的预处理操作包括，旋转、裁剪、归一化等：
- `Resize`：调整图像大小；
- `CenterCrop`：中心裁剪；
- `RandomCrop`：随机裁剪；
- `RandomHorizontalFlip`：随机水平翻转；
- `ToTensor`：将图像转换为Tensor；
- `Normalize`：对图像进行归一化处理。

这些数据增强方法可以通过 `torchvision.transforms.Compose` 来组合使用，以保证每个图像在训练书具有不同的变换：

In [5]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # random horizontal flip
    transforms.RandomRotation(30),      # random rotation
    transforms.RandomResizedCrop(128, scale=(0.8, 1.0)),  # random resized crop
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # color jitter
    transforms.ToTensor(),  # convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # normalization
])

## 4. 加载图像数据集
对于图像数据集，pytorch提供了`torchvision.datasets`模块来加载常用的标准数据集，如MNIST、CIFAR-10、ImageNet等。可以使用`torchvision.datasets`中的类来加载这些数据集，并结合`DataLoader`进行批量加载。

如下加载MNIST数据集的示例：

In [4]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# define the preprocess operations
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # normalize the data
])

# download and load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# create DataLoader for MNIST dataset
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# interate and train the dataset
for inputs, labels in train_loader:
    # Here you can add your training code
    print("Batch inputs shape:", inputs.shape)
    print("Batch labels shape:", labels.shape)

100.0%
100.0%
100.0%
100.0%


Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shape: torch.Size([64])
Batch inputs shape: torch.Size([64, 1, 28, 28])
Batch labels shap

- `datasets.MNIST()`：自动下载并加载MNIST数据集；
- `transforms`参数：指定数据预处理和增强操作---预处理；
- `train`参数：指定加载训练集（`train=True`）或测试集（`train=False`）；

## 5. 使用多个数据源（Multi-Source）
在实际应用中，可能需要处理来自不同来源的数据集。可以通过组合多个 `Dataset` 实例来实现这一点。可以使用 `torch.utils.data.ConcatDataset` 来将多个数据集连接在一起，或者使用 `torch.utils.data.Subset` 来从一个数据集中选择子集。

可以通过继承 `Dataset` 类自定义加载多个数据源，pytorch提供了`ConcatDataset`和`ChainDataset`等类来连接多个数据集。
- `ConcatDataset`：将多个数据集连接在一起，形成一个新的数据集；
- `ChainDataset`：将多个数据集按顺序连接在一起，形成一个新的数据集；
- `Subset`：从一个数据集中选择子集，形成一个新的数据集；
- `random_split`：随机划分数据集为多个子集；
- `SubsetRandomSampler`：从数据集中随机选择样本；

假设有多个图像文件夹的数据，可以将它们合并为一个数据集进行训练。以下是一个简单的示例：

In [6]:
from torch.utils.data import ConcatDataset

dataset1 = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataset2 = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

# assume we have dataset1 and dataset2, which are two Dataset objects
combined_dataset = ConcatDataset([dataset1, dataset2])

# create DataLoader for the combined dataset
combined_loader = DataLoader(combined_dataset, batch_size=64, shuffle=True)

100.0%
100.0%
100.0%
100.0%
